In [106]:
# Johannes Mellak investigating a different sign depending on whether using exact diag. or NetKet expectation values

import netket as nk
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

# --------------------------------------------------------------------------------------------------------
# Prepare the basics (From NetKet/Examples/DissipativeIsing1d)

N = 1

hi = nk.hilbert.Spin(s=1 / 2, N=N)
ha = nk.operator.LocalOperator(hi, dtype=complex)
j_ops = []

lop = 0
I = np.eye(hi.n_states)
# Observables
obs_sy = nk.operator.LocalOperator(hi, dtype=complex)

gp = 0.3
Vp = 2.0
for i in range(N):
    ha += (gp / 2.0) * nk.operator.spin.sigmax(hi, i)
    ha += gp * nk.operator.spin.sigmay(hi, i)
    if N > 1:
        ha += (
            (Vp / 4.0)
            * nk.operator.spin.sigmaz(hi, i)
            * nk.operator.spin.sigmaz(hi, (i + 1) % 2)
        )
    # sigma_{-} dissipation on every site
    #j_ops.append(nk.operator.spin.sigmam(hi, i))
    j_ops.append(1j*nk.operator.spin.sigmam(hi, i))

    obs_sy += nk.operator.spin.sigmay(hi, i)

#j_ops = []
ha = nk.operator.LocalOperator(hi, dtype=complex)
lind = nk.operator.LocalLiouvillian(ha, j_ops)

In [118]:
lop = 0
#lop += -1j*np.kron(ha.to_dense(), I) + 1j*np.kron(I, ha.to_dense())

lop += -1j*np.kron(lind.hamiltonian_nh.to_dense(), I) + 1j*np.kron(I, lind.hamiltonian_nh.to_dense().conj())

for i in range(len(j_ops)):
    L = j_ops[i]
    lop += np.kron(L.to_dense(), L.to_dense().conj())

In [121]:
lop = 0
lop += -1j*np.kron(ha.to_dense(), I) + 1j*np.kron(I, ha.to_dense())

for i in range(len(j_ops)):
    L = j_ops[i]
    LL = L.to_dense().conj().T@L.to_dense()
    lop -= 0.5*(np.kron(LL,I) + np.kron(I,LL.conj()))
    lop += np.kron(L.to_dense(), L.to_dense().conj())

In [122]:
lop

array([[-1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j, -0.5+0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j],
       [ 1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j]])

In [123]:
lind.to_dense()

array([[-1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j, -0.5+0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j],
       [ 1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j]])

In [72]:
np.kron(I,LL)

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0]])

In [73]:
hi.all_states()
lind.hilbert.all_states()

array([[-1, -1],
       [-1, 1],
       [1, -1],
       [1, 1]])

In [74]:
lind._xrv.shape

(6, 1)

In [75]:
v = lind.hilbert.all_states()[3]
print(v)
lind.get_conn(v)

[1 1]


(array([[1, 1],
        [1, 1],
        [1, 1],
        [1, -1],
        [-1, 1],
        [-1, -1]]),
 array([0.-0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]))

In [76]:
lind.hilbert.all_states()

array([[-1, -1],
       [-1, 1],
       [1, -1],
       [1, 1]])

In [81]:
lop

array([[-1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j, -0.5+0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j],
       [ 1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j]])

In [79]:
lind.to_dense()

array([[-1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j, -0.5+0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j],
       [ 1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j]])

In [2]:
# --------------------------------------------------------------------------------------------------------
# Run the optimization

ma = nk.models.NDM(beta=1)
sa = nk.sampler.MetropolisLocal(hilbert=lind.hilbert)
optim = nk.optimizer.Sgd(learning_rate=0.04)

rho = nk.variational.MCMixedState(sampler=sa, model=ma, n_samples=500, n_samples_diag=500)
rho.init_parameters(nk.nn.initializers.normal(stddev=0.01))

ss = nk.SteadyState(lindbladian=lind, optimizer=optim, variational_state=rho)

ss.run(n_iter=300, out='results/testOp', obs={"Sy": obs_sy}) # converges in my simulations

100%|██████████| 300/300 [00:07<00:00, 42.64it/s, LdagL=0.0083 ± 0.0073 [σ²=0.0285]]   


(<netket.logging.json_log.JsonLog at 0x13e71f2e0>,)

In [4]:
# import visualizer        # check convergence
# vis = visualizer.Visualizer('results/testOp')
# vis.plot_observable("LdagL")
# vis.plot_observable("Sy")

# --------------------------------------------------------------------------------------------------------
# Compare the results for some observable

# The observable operator to investigate
obs = obs_sy

# Calculate exact result
rho_0 = nk.exact.steady_state(lind)

print('_'*80)
print(f'RBM result for the observable: {ss.estimate(obs)}')
print(f'Exact result from ED:          {np.trace(rho_0@obs.to_dense())}')

print('...these results have a different sign. (High uncertainty is due to the imaginary part)')
print('_'*80)



Minimum eigenvalue is:  -2.0746007728622543e-16
________________________________________________________________________________
RBM result for the observable: -0.064+0.036j ± 0.050 [σ²=1.266, R̂=0.9943]
Exact result from ED:          (0.08141487086313592+2.914335439641036e-16j)
...these results have a different sign. (High uncertainty is due to the imaginary part)
________________________________________________________________________________


In [40]:
print(rho_0.imag)

[[ 4.48231214e-17  1.55240220e-03  1.55240220e-03  2.06986960e-02]
 [-1.55240220e-03  7.29129204e-17  8.98873569e-17  1.88013155e-02]
 [-1.55240220e-03 -4.51365698e-17 -1.96749150e-17  1.88013155e-02]
 [-2.06986960e-02 -1.88013155e-02 -1.88013155e-02 -9.87603578e-17]]


In [44]:
rho_0 = nk.exact.steady_state(lind, method="iterative")
print(rho_0.imag)

Starting iterative solver...
Converged trace is  (0.9999999999999998+1.776677594546218e-16j)
[[-5.32606981e-13  1.55238327e-03  1.55238327e-03  2.06986474e-02]
 [-1.55238326e-03  5.45092423e-12  6.30289646e-12  1.88016999e-02]
 [-1.55238326e-03  6.30286644e-12  5.45092515e-12  1.88016999e-02]
 [-2.06986474e-02 -1.88016998e-02 -1.88016998e-02 -1.03690647e-11]]


In [ ]:



# --------------------------------------------------------------------------------------------------------
# Digging deeper into the code of netket... (DEBUG CODE)
# 1.) I reproduce the code that calculates the expectation value and
# 2.) replace the RBM result of the density matrix rho with the exact rho_0
#
# => the expectation value gives the exact result with a negative sign for asymmetric observables (e.g. S_y)

def reproduceLocalValueCalculation(samples_u, operator, use_exact_rho=False):
    # Reproduce local value calculation from _local_cost_functions.py::198:local_value_op_op_cost()

    samples_uc, mels = operator.get_conn_padded(samples_u)

    import jax
    import jax.numpy as jnp

    locs = []
    for batch in range(samples_u.shape[0]):
        # Manually handling batches
        s = samples_u[batch]
        s_ind = hi.states_to_numbers(s)
        s_p = samples_uc[batch]
        p_ind = hi.states_to_numbers(s_p)
        mel = mels[batch]

        # from local_value_op_op_cost
        s_sp = jax.vmap(lambda samples, samples_p: jnp.hstack((samples, samples_p)), in_axes=(None, 0))(s, s_p)
        s_s = jnp.hstack((s, s))

        # Get the RBM values
        log_s_sp = rho._apply_fun(rho.variables, s_sp)
        log_s_s = rho._apply_fun(rho.variables, s_s)
        # Or better: Use the exact rho values from ED instead of the function values
        if use_exact_rho:
            log_s_sp = np.log(rho_0[s_ind][p_ind])
            log_s_s = np.log(rho_0[s_ind][s_ind])

        # Calculate local value
        locs.append(jnp.sum(mel * jnp.exp(log_s_sp - log_s_s)))

    return locs

# MCMC samples that were generated by the sampler
samples_r = np.asarray(rho.diagonal.samples.reshape((-1, rho.diagonal.samples.shape[-1])))

print('Calculating expectation value "manually" ... takes a minute')
locs_r = reproduceLocalValueCalculation(samples_r, obs)
print('_'*80)
print(f'Reproduction of expectation value: {np.mean(locs_r)}')
print(f'Netket expectation value:          {ss.estimate(obs).mean}')

print('check: the reproduction of the netket code works')

# to remove MC noise, uniform distribution of states
samples_uniform = hi.all_states()
locs_u = reproduceLocalValueCalculation(samples_uniform, obs, use_exact_rho=True)
# Correct the probability distribution of the uniform samples according to the diagonal of rho
result_u = np.sum(np.array(locs_u) * rho_0.diagonal())

print('_'*80)
print(f'Reproduction of expectation value using rho_0: {result_u}')
print(f'Result from exact diagonalization rho_0      : {np.trace(rho_0@obs.to_dense())}')
print('...again, these results have a different sign')

print('_'*80)
print('finished')

In [8]:
rho.expect(obs)

0.052+0.075j ± 0.038 [σ²=0.744, R̂=0.9962]

In [13]:
np.trace(rho.to_matrix().T@obs.to_dense())

(0.06881311028043385+1.3877787807814457e-17j)

In [15]:
import qutip

In [18]:
obs_sx = sum([nk.operator.spin.sigmax(hi, i) for i in range(2)])
obs_sz = sum([nk.operator.spin.sigmaz(hi, i) for i in range(2)])

In [21]:
rho.expect(obs_sx)

0.225-0.017j ± 0.038 [σ²=0.757, R̂=0.9962]

In [23]:
rho.expect(obs_sy)

0.052+0.075j ± 0.038 [σ²=0.744, R̂=0.9962]

In [22]:
rho.expect(obs_sz)

-1.9844+0.0000j ± 0.0078 [σ²=0.0310, R̂=0.9962]

In [28]:
ha.to_dense()

array([[ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [48]:
nk.operator.spin.sigmax(hi,1).to_dense()

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [47]:
nk.operator.spin.sigmay(hi,1).to_dense()

array([[0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j]])

In [63]:
(lind.to_dense().imag)

array([[0, -0.15, -0.15, 0, 0.15, 0, 0, 0, 0.15, 0, 0, 0, 0, 0, 0, 0],
       [-0.15, 2, 0, -0.15, 0, 0.15, 0, 0, 0, 0.15, 0, 0, 0, 0, 0, 0],
       [-0.15, 0, 2, -0.15, 0, 0, 0.15, 0, 0, 0, 0.15, 0, 0, 0, 0, 0],
       [0, -0.15, -0.15, 0, 0, 0, 0, 0.15, 0, 0, 0, 0.15, 0, 0, 0, 0],
       [0.15, 0, 0, 0, -2, -0.15, -0.15, 0, 0, 0, 0, 0, 0.15, 0, 0, 0],
       [0, 0.15, 0, 0, -0.15, 0, 0, -0.15, 0, 0, 0, 0, 0, 0.15, 0, 0],
       [0, 0, 0.15, 0, -0.15, 0, 0, -0.15, 0, 0, 0, 0, 0, 0, 0.15, 0],
       [0, 0, 0, 0.15, 0, -0.15, -0.15, -2, 0, 0, 0, 0, 0, 0, 0, 0.15],
       [0.15, 0, 0, 0, 0, 0, 0, 0, -2, -0.15, -0.15, 0, 0.15, 0, 0, 0],
       [0, 0.15, 0, 0, 0, 0, 0, 0, -0.15, 0, 0, -0.15, 0, 0.15, 0, 0],
       [0, 0, 0.15, 0, 0, 0, 0, 0, -0.15, 0, 0, -0.15, 0, 0, 0.15, 0],
       [0, 0, 0, 0.15, 0, 0, 0, 0, 0, -0.15, -0.15, -2, 0, 0, 0, 0.15],
       [0, 0, 0, 0, 0.15, 0, 0, 0, 0.15, 0, 0, 0, 0, -0.15, -0.15, 0],
       [0, 0, 0, 0, 0, 0.15, 0, 0, 0, 0.15, 0, 0, -0.15, 2, 0, -0.15],
  

In [69]:
a=nk.exact.steady_state(lind).reshape((-1,1))
print(a)

Minimum eigenvalue is:  -1.6487874695618348e-16
[[ 4.65720660e-04-1.17605414e-16j]
 [-7.02888208e-17+1.55240220e-03j]
 [ 1.48013739e-16+1.55240220e-03j]
 [-5.17467400e-03+2.06986960e-02j]
 [-1.39515342e-16-1.55240220e-03j]
 [ 5.64039466e-03+1.71384329e-16j]
 [ 5.17467400e-03+1.87672922e-17j]
 [ 6.89956533e-02+1.88013155e-02j]
 [-1.93515570e-16-1.55240220e-03j]
 [ 5.17467400e-03-7.15281703e-17j]
 [ 5.64039466e-03+1.79174526e-16j]
 [ 6.89956533e-02+1.88013155e-02j]
 [-5.17467400e-03-2.06986960e-02j]
 [ 6.89956533e-02-1.88013155e-02j]
 [ 6.89956533e-02-1.88013155e-02j]
 [ 9.88253490e-01-3.62598250e-16j]]


In [23]:
hi1 = nk.hilbert.Spin(0.5)
sx = nk.operator.spin.sigmax(hi1, 0)
sy = nk.operator.spin.sigmay(hi1, 0)
sz = nk.operator.spin.sigmaz(hi1, 0)

In [31]:
nk.hilbert.Spin(0.5,2).all_states()

array([[-1, -1],
       [-1, 1],
       [1, -1],
       [1, 1]])

In [24]:
sm = (0.5*(sx-1j*sy))
sm.to_dense()

array([[0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j]])

In [33]:
np.kron(sm.to_dense(), sm.to_dense().T)

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [107]:
v = hi1.all_states()[0]
print(v)
print(sm.get_conn(v))

v = hi1.all_states()[1]
print(v)
print(sm.get_conn(v))

v = hi1.all_states()[1]
print(v)
print(sz.get_conn(v))

[0]
(array([[0],
       [-1]]), array([0.+0.j, 1.+0.j]))
[1]
(array([[1],
       [-1]]), array([0.+0.j, 1.+0.j]))
[1]
(array([[1]]), array([-1]))


In [151]:
smm = nk.operator.spin.sigmam(hi1, 0)
smm._mels

array([[[nan],
        [1]]])

In [142]:
sy._acting_size
sy._local_states
sy._x_prime

array([[[[1]],

        [[-1]]]])

In [65]:
def _number_to_state(number, local_states, out):

    out.fill(local_states[0])
    size = out.shape[0]
    local_size = local_states.shape[0]

    ip = number
    k = size - 1
    while(ip > 0):
        out[k] = local_states[ip % local_size]
        ip = ip // local_size
        k -= 1

    return out

def _number_to_state2(number, hilbert_size_per_site, local_states_per_site, out):

    out[:] = local_states_per_site[:, 0]
    size = out.shape[0]

    ip = number
    k = size - 1
    while ip > 0:
        local_size = hilbert_size_per_site[k]
        out[k] = local_states_per_site[k, ip % local_size]
        ip = ip // local_size
        k -= 1

    return out


In [69]:
_number_to_state(0, np.array([-1,1]), np.empty(1))

array([-1])

In [70]:
_number_to_state2(0, np.array([2]), np.array([[-1,1]]), np.empty(1))

array([-1])

In [97]:
des.to_dense()

array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1.41, 0, 0],
       [0, 0, 1.73, 0]])

In [105]:
v = hi1.all_states()[0]
print(v)
print(des.get_conn(v))

v = hi1.all_states()[1]
print(v)
print(des.get_conn(v))

v = hi1.all_states()[2]
print(v)
print(des.get_conn(v))


[0]
(array([[0]]), array([0]))
[1]
(array([[1],
       [0]]), array([0, 1]))
[2]
(array([[2],
       [1]]), array([0, 1.41]))


In [106]:
sm.to_dense()

array([[0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j]])

In [76]:
sm.get_conn(v)

(array([[1],
        [-1]]),
 array([0, 1]))

In [102]:
g = nk.graph.Chain(2, pbc=False)
hi2 = nk.hilbert.Spin(0.5, 2)
sm0 = nk.operator.spin.sigmam(hi2, 0)
sp0 = nk.operator.spin.sigmap(hi2, 0)
sm1 = nk.operator.spin.sigmam(hi2, 1)
sp1 = nk.operator.spin.sigmap(hi2, 1)

hi = nk.operator.Ising(hi2, graph=g, h=0.0)